In [1]:
# Loading packages
!pip install nltk

exec(open('utilities_cw.py').read())

# We fix the seeds to get consistent results
SEED = 1111
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
one.py is being run directly


In [0]:
from sklearn.model_selection import train_test_split

train_file = 'offenseval-training-v1.tsv'
corpus, labels = parse_dataset_task_b(train_file)

In [4]:
print(len(corpus))
print(len(labels))


4400
4400


In [0]:
test_file = 'validation.tsv'
test_corpus, test_labels = parse_dataset_task_b(test_file)

In [0]:
corpus += test_corpus
labels += test_labels

In [7]:
train_corpus, valid_corpus, train_labels, valid_labels = train_test_split(corpus, labels, test_size=0, random_state=42)

train_corpus, train_labels = augment_untargeted(train_corpus, train_labels, 1600)
# valid_corpus, valid_labels = augment_untargeted(valid_corpus, valid_labels, 900)
    
print(train_corpus)
print(train_labels)

print(len(train_corpus))
print(len(train_labels))

['user ur so fuckidng pretty perhaps i shed tears url', 'user he is toxic even for alligators', 'user user shes tragic because she is an idiot and because she had tremendous internalized misogyny   i think of the moment we are in the grips of as the tyranny of the halfidiot', 'user this is so entertaining nobody has thought about montel in years and you are picking a fight with him this is incredible such a feeble mind you have', 'user goodell is one of the worst things that has ever been unleashed on pro sports  useless', 'user maga their podcastful is about blockchainful but itsful about what it meansful to bewise working mothersful who are trying to makewise', 'user he is bafoonicus ', 'user user you go mike and by the way sarah she is the best lair  url', 'user user laughing out loud another illegal antifa girl', 'anxiety fucking sucks ', 'user a dying sport', 'user gonna need a whole ass rant thread done in the utara loghat to really feel it haha', 'user user user lindsey sean han

In [0]:
tokenize_f = tokenize_stemming

In [9]:
tokenized_train_corpus = tokenize_f(train_corpus)
print(len(tokenized_train_corpus))

6077


In [10]:
vocabulary = get_vocabulary(tokenized_train_corpus)
print(vocabulary)
print(len(vocabulary))

['user', 'ur', 'fuckidng', 'pretti', 'perhap', 'shed', 'tear', 'url', 'toxic', 'even', 'allig', 'she', 'tragic', 'idiot', 'tremend', 'intern', 'misogyni', '', 'think', 'moment', 'grip', 'tyranni', 'halfidiot', 'entertain', 'nobodi', 'thought', 'montel', 'year', 'pick', 'fight', 'incred', 'feebl', 'mind', 'goodel', 'one', 'worst', 'thing', 'ever', 'unleash', 'pro', 'sport', 'useless', 'maga', 'podcast', 'blockchain', 'its', 'means', 'bewis', 'work', 'mothers', 'tri', 'makewis', 'bafoonicu', 'go', 'mike', 'way', 'sarah', 'best', 'lair', 'laugh', 'loud', 'anoth', 'illeg', 'antifa', 'girl', 'anxieti', 'fuck', 'suck', 'die', 'gonna', 'need', 'whole', 'ass', 'rant', 'thread', 'done', 'utara', 'loghat', 'realli', 'feel', 'haha', 'lindsey', 'sean', 'hanniti', 'talk', 'echo', 'chamber', 'simpli', 'hear', 'journalist', 'news', 'pundit', 'weird', 'conspiratori', 'agenda', 'support', 'well', 'craft', 'partial', 'fact', 'aim', 'narrow', 'audienc', 'laughabl', 'fool', 'trump', 'im', 'screw', 'dream'

In [11]:
word2idx = get_word2idx(tokenized_train_corpus, vocabulary)
print(len(word2idx))

8287


In [12]:
idx2word = get_idx2word(vocabulary)
print(len(idx2word))

8286


In [13]:
sentences_lengths = [len(sentence) for sentence in tokenized_train_corpus]
max_len = np.max(np.array(sentences_lengths))

train_sentences_tensor, train_labels_tensor = parse_input(tokenized_train_corpus, word2idx, train_labels, max_len)

print(train_sentences_tensor)
print(train_labels_tensor)

print(train_sentences_tensor.shape)
print(train_labels_tensor.shape)

tensor([[   1,    2,    3,  ...,    0,    0,    0],
        [   1,    9,   10,  ...,    0,    0,    0],
        [   1,    1,   12,  ...,    0,    0,    0],
        ...,
        [   1, 2331,    0,  ...,    0,    0,    0],
        [6117, 1319, 5454,  ...,    0,    0,    0],
        [   1, 5821,  260,  ...,    0,    0,    0]])
tensor([1., 1., 1.,  ..., 0., 0., 0.])
torch.Size([6077, 81])
torch.Size([6077])


In [0]:
def parse_in(tokenized_corpus, word2idx, labels, max_len):
    # we index our sentences
    vectorized_sentences = [[word2idx[token] for token in sentence if token in word2idx] for sentence in tokenized_corpus]
  
  
    # we create a tensor of a fixed size filled with zeroes for padding
    sentences_tensor = Variable(torch.zeros((len(vectorized_sentences), max_len))).long()
    sentences_lengths = [len(sentence) for sentence in vectorized_sentences]

    # we fill it with our vectorized sentences 
    for idx, (sentence, sentence_len) in enumerate(zip(vectorized_sentences, sentences_lengths)):
        sentences_tensor[idx, :sentence_len] = torch.LongTensor(sentence)
            
    labels_tensor = torch.FloatTensor(labels)

    return sentences_tensor, labels_tensor

In [0]:
def train_model_custom(model, optimizer, loss_fn, feature_train, target_train):    
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        predictions = model(feature_train).squeeze(1)
        loss = loss_fn(predictions, target_train)
        acc = accuracy(predictions, target_train)
        loss.backward()
        optimizer.step()

        epoch_loss = loss.item()
        epoch_acc = acc
  
        print(f'| Epoch: {epoch:02} | Train Loss: {epoch_loss:.3f} | Train Acc: {epoch_acc*100:.2f}% | ')


In [16]:
epochs=100

INPUT_DIM = len(word2idx)
EMBEDDING_DIM = 100
OUTPUT_DIM = 1
LEARNING_RATE = 0.0008

# the hyperparamerts specific to CNN
# we define the number of filters
N_OUT_CHANNELS = 100
# we define the window size
WINDOW_SIZE = 2
# we apply the dropout with the probability 0.5
DROPOUT = 0.5

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_OUT_CHANNELS, WINDOW_SIZE, OUTPUT_DIM, DROPOUT)

optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
loss_fn = nn.BCEWithLogitsLoss()

train_model_custom(model, optimizer, loss_fn, train_sentences_tensor, train_labels_tensor)

| Epoch: 00 | Train Loss: 0.678 | Train Acc: 59.39% | 
| Epoch: 01 | Train Loss: 0.675 | Train Acc: 60.57% | 
| Epoch: 02 | Train Loss: 0.667 | Train Acc: 61.81% | 
| Epoch: 03 | Train Loss: 0.659 | Train Acc: 62.66% | 
| Epoch: 04 | Train Loss: 0.644 | Train Acc: 65.02% | 
| Epoch: 05 | Train Loss: 0.639 | Train Acc: 64.79% | 
| Epoch: 06 | Train Loss: 0.637 | Train Acc: 64.95% | 
| Epoch: 07 | Train Loss: 0.628 | Train Acc: 65.23% | 
| Epoch: 08 | Train Loss: 0.624 | Train Acc: 65.71% | 
| Epoch: 09 | Train Loss: 0.617 | Train Acc: 66.88% | 
| Epoch: 10 | Train Loss: 0.616 | Train Acc: 66.51% | 
| Epoch: 11 | Train Loss: 0.612 | Train Acc: 67.19% | 
| Epoch: 12 | Train Loss: 0.601 | Train Acc: 67.96% | 
| Epoch: 13 | Train Loss: 0.600 | Train Acc: 68.44% | 
| Epoch: 14 | Train Loss: 0.592 | Train Acc: 68.82% | 
| Epoch: 15 | Train Loss: 0.583 | Train Acc: 69.46% | 
| Epoch: 16 | Train Loss: 0.588 | Train Acc: 69.89% | 
| Epoch: 17 | Train Loss: 0.580 | Train Acc: 70.10% | 
| Epoch: 1

In [0]:
testing = read_csv('testset-taskb.tsv')

tweets = [row['tweet'] for row in testing]
tokenized_corpus = tokenize(tweets)

test_tensor, labels_tensor = parse_input(tokenized_corpus, word2idx, test_labels, max_len)

In [21]:
predictions_test = model(test_tensor).squeeze(1)

output = torch.round(torch.sigmoid(predictions_test))

print(output)

print(len(output))

tensor([0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1.,
        1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1., 0.,
        0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1.,
        1., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 1.,
        0., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0.,
        0., 1., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 0., 0., 1., 0., 1., 1.,
        1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 0.,
        1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0.,
        1., 0., 1., 0., 0., 1., 1., 1., 0., 1., 1., 0., 0., 0., 1., 1., 1., 0.,
        0., 1., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 1., 0., 0.,
        0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0.,
        1., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 1., 1.,
        1., 0., 0., 0., 0., 0., 1., 0., 

In [0]:
ids = [row['id'] for row in testing]
out = ['UNT' if e == 0 else 'TIN' for e in output.detach().numpy()]
zipped = list(zip(ids, out))

with open('predictions_task_b_full_training_stemming_window_2_augmentation.csv', "w") as f:
    writer = csv.writer(f, dialect='excel')
    for row in zipped:
        writer.writerow(row)